# Data Analysis

## Import related packages

In [1]:
import numpy as np
import pandas as pd

## Load datasets
**NOTE**: Because the size of original data files is too large to upload all of them to GitHub, so you cannot run the data command directly. However, I put the link of datasets in ```READ.md```and you can run the cell after downloading the data. 

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.head()

,Id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Ret_175,Ret_176,Ret_177,Ret_178,Ret_179,Ret_180,Ret_PlusOne,Ret_PlusTwo,Weight_Intraday,Weight_Daily
0,1,NaN,NaN,NaN,NaN,8.0,NaN,75751,0.2254,11.0,...,-0.002688,0.002246,-0.000838,-6.953224e-04,0.000003,-0.001974,-0.019512,0.028846,1.251508e+06,1.564385e+06
1,2,NaN,NaN,NaN,NaN,3.0,0.388896,17369,0.0166,13.0,...,-0.000129,0.000123,0.000248,3.315418e-07,0.000003,0.000027,-0.002939,-0.010253,1.733950e+06,2.167438e+06
2,3,NaN,-0.696727,0.739591,-0.167928,9.0,0.471947,8277,0.3650,9.0,...,-0.000524,-0.000394,0.000116,5.322557e-04,0.000274,0.000784,-0.024791,0.015711,1.529197e+06,1.911497e+06
3,4,NaN,-0.694350,1.568248,0.479073,5.0,0.120653,22508,0.2654,13.0,...,0.000346,-0.000090,0.000288,-1.281102e-04,0.000074,0.000341,-0.005680,-0.002190,1.711569e+06,2.139462e+06
4,5,6.0,-1.736489,2.765531,1.245280,7.0,4.866985,22423,0.2138,13.0,...,-0.001235,0.000027,0.002449,8.619882e-06,0.001209,-0.000004,0.036104,-0.026552,1.267270e+06,1.584088e+06


## Construct proper labels
Ret_2 - Ret_120 are return per minute at Day D(intraday). However, the aggregative return at 120 minute is more useful. Hence, we sum these data and caiculate standard deviation. This process can help us simplify the original data and clearly observe them.

In [4]:
# Add new labels
intraday_return = []
Return = ['Ret_MinusTwo', 'Ret_MinusOne']
Train_Agg_Ret = pd.DataFrame(columns=['R_Agg', 'R_Agg_Std', 'R_Std'])
Test_Agg_Ret = pd.DataFrame(columns=['R_Agg', 'R_Agg_Std', 'R_Std'])

for i in range(2, 121):
    intraday_return.append(f'Ret_{i}')
    
# Aggregative return from 2-120 minute
Train_Agg_Ret['R_Agg'] = train_df[intraday_return].sum(axis=1)
Test_Agg_Ret['R_Agg'] = test_df[intraday_return].sum(axis=1)
# Standard deviation
Train_Agg_Ret['R_Agg_Std'] = train_df[intraday_return].std(axis=1)
Test_Agg_Ret['R_Agg_Std'] = test_df[intraday_return].std(axis=1)
Train_Agg_Ret['R_Std'] = train_df[Return].std(axis=1)
Test_Agg_Ret['R_Std'] = test_df[Return].std(axis=1)
# Merge dat
Train_df = pd.concat([train_df, Train_Agg_Ret], axis=1)
Test_df = pd.concat([test_df, Test_Agg_Ret], axis=1)
Train_df.head()
Test_df.head()

,Id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Ret_114,Ret_115,Ret_116,Ret_117,Ret_118,Ret_119,Ret_120,R_Agg,R_Agg_Std,R_Std
0,1,1.0,1.412783,-0.056284,NaN,10.0,0.413226,18871,0.2138,11.0,...,0.000366,-0.002444,-0.001301,-0.000917,0.000762,-0.000556,0.000759,0.001640,0.000808,0.034740
1,2,NaN,NaN,0.907973,1.002425,7.0,-0.257825,5852,0.2138,13.0,...,-0.000003,-0.000012,-0.000463,-0.000003,-0.000002,0.000468,-0.000012,-0.003270,0.000538,0.000452
2,3,NaN,-0.607583,1.076668,0.517865,5.0,0.947340,76935,0.0105,10.0,...,0.000003,NaN,0.000164,0.000353,0.000704,-0.000168,-0.000006,0.003425,0.000439,0.017476
3,4,NaN,2.230240,0.223222,NaN,1.0,NaN,84573,0.3318,13.0,...,0.000017,0.000170,-0.001079,0.000320,0.000006,0.001392,-0.001850,0.001612,0.000561,0.008162
4,5,NaN,0.360399,0.597896,-0.145497,10.0,0.275744,89615,0.0099,8.0,...,-0.000495,0.000057,-0.000050,-0.000011,-0.000377,0.000227,-0.000114,0.002558,0.000317,0.009252


## Split the orginal data to several parts

In [5]:
Label = ['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 
         'Feature_6','Feature_7', 'Feature_8', 'Feature_9', 'Feature_10', 
         'Feature_11', 'Feature_12','Feature_13', 'Feature_14', 'Feature_15', 
         'Feature_16', 'Feature_17', 'Feature_18','Feature_19', 'Feature_20', 
         'Feature_21', 'Feature_22', 'Feature_23', 'Feature_24','Feature_25', 
         'Ret_MinusTwo', 'Ret_MinusOne', 'R_Agg', 'R_Agg_Std', 'R_Std']
targets = ['Ret_PlusOne', 'Ret_PlusTwo']# D+1 and D+2
weights_intraday = 'Weight_Intraday'
weights_daily = 'Weight_Daily'
weights = [weights_intraday, weights_daily]

X_train = Train_df[Label]
y_train = Train_df[targets]
Train_weights = Train_df[weights_daily]
X_test = Test_df[Label]

print(f'Shape of training label data: {X_train.shape}')
print(f'Shape of training target data: {y_train.shape}')
print(f'Shape of test label data: {X_test.shape}')

Shape of training label data: (40000, 30)
Shape of training target data: (40000, 2)
Shape of test label data: (120000, 30)


In [6]:
X_train.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Ret_MinusTwo,Ret_MinusOne,R_Agg,R_Agg_Std,R_Std
0,NaN,NaN,NaN,NaN,8.0,NaN,75751,0.2254,11.0,NaN,...,NaN,-0.489492,NaN,NaN,NaN,0.055275,-0.010770,-0.025248,0.000826,0.046701
1,NaN,NaN,NaN,NaN,3.0,0.388896,17369,0.0166,13.0,NaN,...,0.886205,-0.151805,1.239856,0.953424,-0.709462,0.009748,0.002987,-0.008142,0.000364,0.004780
2,NaN,-0.696727,0.739591,-0.167928,9.0,0.471947,8277,0.3650,9.0,5.0,...,NaN,0.389061,1.728096,-1.798090,-1.019370,0.003077,0.006181,-0.000633,0.000525,0.002195
3,NaN,-0.694350,1.568248,0.479073,5.0,0.120653,22508,0.2654,13.0,5.0,...,1.148738,-2.440799,1.551425,-1.788725,NaN,0.000984,0.014106,0.001276,0.000294,0.009278
4,6.0,-1.736489,2.765531,1.245280,7.0,4.866985,22423,0.2138,13.0,4.0,...,3.381040,1.732708,1.965189,-5.993092,3.219820,-0.018224,0.011065,-0.002293,0.000832,0.020711


In [7]:
y_train.head()

,Ret_PlusOne,Ret_PlusTwo
0,-0.019512,0.028846
1,-0.002939,-0.010253
2,-0.024791,0.015711
3,-0.005680,-0.002190
4,0.036104,-0.026552


## Analyzing the the nature of features
Obviously, we can find that some data are integer and some others are decimal from the ```.csv``` files. Apart from these, some data are missing. In order to better use these data, we will try to understand these features by a basic analysis.

In [9]:
# Test singular values by 3-sigma principle
def Singular(feature):
    std_test_3 = feature.std()*3
    mean_test = feature.mean()
    Number_singular = 0
    for val in feature:
        if (abs(val-mean_test) > std_test_3):
            Number_singular = Number_singular + 1
    return Number_singular

# Test the number and percentage of missing data, unique data, special data and singualr data
def Test(item,df,percentage=True):
    vals = []
    vals_percentage = []
    for col in df:
        item_switch = {
                'Missing': sum(df[col].isnull()),
                'Unique': len(df[col].unique()),
                'Special': df[col].value_counts().values[0],
                'Singular': Singular(df[col])}
        val = item_switch.get(item)
        vals.append(val)
        vals_percentage.append(val/len(df[col])*100)
    if percentage:
        df_final = pd.DataFrame(list(zip(vals,vals_percentage)),columns=[item,f'{item} %'])
    else:
        df_final = pd.DataFrame(list(zip(vals)),columns=[item])
    return df_final

In [10]:
missing = Test('Missing',X_train)
unique = Test('Unique', X_train)
special = Test('Special', X_train)
singular = Test('Singular', X_train)  
Analysis_Results = pd.concat([pd.DataFrame(X_train.columns), missing, unique,
                              special, singular], axis=1)

In [12]:
Analysis_Results    

,0,Missing,Missing %,Unique,Unique %,Special,Special %,Singular,Singular %
0,Feature_1,33313,83.2825,11,0.0275,2651,6.6275,0,0.0000
1,Feature_2,9146,22.8650,30855,77.1375,1,0.0025,62,0.1550
2,Feature_3,1237,3.0925,38764,96.9100,1,0.0025,269,0.6725
3,Feature_4,7721,19.3025,32280,80.7000,1,0.0025,80,0.2000
4,Feature_5,0,0.0000,10,0.0250,6943,17.3575,0,0.0000
5,Feature_6,1933,4.8325,38068,95.1700,1,0.0025,1018,2.5450
6,Feature_7,0,0.0000,824,2.0600,114,0.2850,0,0.0000
7,Feature_8,469,1.1725,33,0.0825,4178,10.4450,0,0.0000
8,Feature_9,1875,4.6875,37,0.0925,5863,14.6575,237,0.5925
9,Feature_10,19471,48.6775,7,0.0175,14437,36.0925,778,1.9450
